In [27]:
import pandas as pd

from xgboost import XGBRegressor

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt

In [28]:
import pandas as pd

F_selection = pd.read_csv("F_Selection/top_N_features_PersonsRAll.csv")

# Extract top 10, 20, 30, and 50 features
top_10 = F_selection["Feature"].head(10).tolist()
top_20 = F_selection["Feature"].head(20).tolist()
top_30 = F_selection["Feature"].head(30).tolist()
top_40 = F_selection["Feature"].head(40).tolist()
top_50 = F_selection["Feature"].head(50).tolist()

In [29]:
def Data_Organizer(raw_Data,top):
    
    All_players_dataSet = raw_Data.copy()
    
    All_players_dataSet = All_players_dataSet.drop(columns=All_players_dataSet.columns[All_players_dataSet.columns.str.contains('Unnamed:')])
    All_players_dataSet.reset_index(drop=True, inplace=True)
    
    All_players_dataSet.fillna(0, inplace=True)
    All_players_dataSet.replace('', 0, inplace=True)
    All_players_dataSet.replace('--', 0, inplace=True)
    
    # check if any NaN,empty Strings exists in the dataframe
    any_missing_values = All_players_dataSet.isna().any().any()
    any_empty_values = (All_players_dataSet.applymap(lambda x: x == '')).any().any()
    
    if any_missing_values or any_empty_values:
        print("DataFrame contains missing values or empty strings/spaces.")
    else:
        print("DataFrame does not contain missing values or empty strings/spaces.")
        
    ## seperating our dataFrame
    Y = All_players_dataSet['Points_won']  # Target
    X = All_players_dataSet.drop('Points_won', axis=1)  # Features
    
    X = X.apply(pd.to_numeric, errors='coerce')

    ## keep the specified columns
    columns_to_keep = top
    
    ## keep only wanted columns
    X = X[columns_to_keep]

    return X, Y

In [30]:
all_players_w_add_sorted = pd.read_csv("all_players_w_add_sorted.csv")

C:\Users\Abdou\AppData\Local\Temp\ipykernel_10192\1174582640.py:1: DtypeWarning: Columns (154,155) have mixed types. Specify dtype option on import or set low_memory=False.
  all_players_w_add_sorted = pd.read_csv("all_players_w_add_sorted.csv")


In [31]:
top = top_10

rank_data_10 = pd.DataFrame()
    
for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a XGBoost Regressor model
    xgb_model = XGBRegressor(n_estimators=200,max_depth = 5, random_state=42, n_jobs=-1)
    xgb_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_xgb = xgb_model.predict(X_test)
    
    # Evaluate the model
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (XGBoost): {mse_xgb}')
    print(f'R-squared (XGBoost): {r2_xgb}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_xgb
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)
    
    if rank_data_10.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_10 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_10 = pd.concat([rank_data_10.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_10)

Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 16.110317865729826
R-squared (XGBoost): -1.588330175861226


,Player_name,Rank,Pos,Points_won,Predicted_points
13740,Hakeem Olajuwon,-1,C,0.0,72.997696
13764,Gary Payton,2,PG,37.0,72.997696
13894,Dikembe Mutombo,1,C,39.0,72.997696
13836,Michael Jordan,4,SG,6.0,8.998991
13631,Derrick McKey,-1,SF,0.0,2.999332
13852,Patrick Ewing,-1,C,0.0,2.999332
13705,Shawn Kemp,-1,PF,0.0,2.999332
13783,Mario Elie,-1,SF,0.0,2.772643
13675,Nate McMillan,-1,SG,0.0,2.772643
13555,Latrell Sprewell,-1,SG,0.0,2.772643


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 11.139616446291527
R-squared (XGBoost): 0.34416885324166613


,Player_name,Rank,Pos,Points_won,Predicted_points
13269,Alonzo Mourning,1,C,89.0,18.470200
13182,Dikembe Mutombo,2,C,10.0,6.410304
13488,Tim Duncan,5T,PF,2.0,6.346099
13434,David Robinson,4,C,3.0,5.920939
13406,Dennis Rodman,-1,PF,0.0,5.902582
13050,Charles Oakley,-1,PF,0.0,5.902582
13087,Horace Grant,-1,PF,0.0,5.899345
13121,Bo Outlaw,-1,PF,0.0,5.899345
13397,Scottie Pippen,7T,SF,1.0,5.783936
13445,Kevin Garnett,7T,PF,1.0,5.783197


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 6.3484964515239835
R-squared (XGBoost): 0.34524406938871366


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1007,Joel Embiid,-1,C,0.0,99.466621
1022,Rudy Gobert,3.0,C,136.0,26.091211
1026,Myles Turner,-1.0,C,0.0,25.580944
933,Al Horford,9,C,3.0,13.689749
678,Chris Paul,-1,PG,0.0,13.676902
968,Jrue Holiday,8,PG,6.0,11.925324
1111,Giannis Antetokounmpo,6.0,PF,58.0,9.569980
877,Kyle Lowry,-1,PG,0.0,9.130501
713,Marcus Smart,1,PG,257.0,9.095054
997,Jaren Jackson Jr.,5.0,PF,99.0,7.759455


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 83.06949272028258
R-squared (XGBoost): 0.8047799232715728


,Player_name,Rank,Pos,Points_won,Predicted_points
59,Jaren Jackson Jr.,1.0,C,391.0,356.277252
323,Brook Lopez,2.0,C,309.0,126.946243
206,Nic Claxton,9T,C,7.0,80.977028
436,Draymond Green,4,PF,34.0,17.596373
524,Bam Adebayo,5.0,C,18.0,12.193107
312,LeBron James,-1,PF,0.0,11.983219
497,Joel Embiid,9T,C,7.0,9.528023
504,Paul George,-1.0,SF,0.0,6.874328
484,Chris Paul,-1,PG,0.0,6.629869
531,Mikal Bridges,-1.0,SF,0.0,5.429190


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,-1,1,1,-1,1,-1,2,1,1,-1,...,1.0,10T,3,-1.0,1.0,6T,1.0,1.0,-1,1.0
1,2,2,8T,-1,-1,11,1,4,8,3,...,13T,8,1,-1.0,3,2.0,3.0,9.0,3.0,2.0
2,1,5T,8T,-1,3T,11,-1,5,10T,24T,...,8T,13T,-1,-1,15T,1.0,11,5.0,-1.0,9T
3,4,4,7,-1,-1,1,7,9,-1,8,...,7.0,6.0,5.0,-1,6,-1,2,-1,9,4
4,-1,-1,-1,1,-1,13T,6,-1,7,4,...,10.0,1,2,3,-1,-1,8T,3,-1,5.0
5,-1,-1,2,1,-1,3,-1,8,13T,13T,...,4.0,4,-1.0,2.0,-1.0,5.0,10.0,-1,8,-1
6,-1,-1,8T,-1,-1,4,4,18T,-1,1,...,5,7.0,11T,1,-1,-1.0,-1,7,6.0,9T
7,-1,-1,3T,8T,-1,-1,10T,-1,3,5,...,6,5.0,-1.0,-1,-1,-1,-1.0,-1.0,-1,-1.0
8,-1,7T,8T,-1,2,2,8,-1,4,-1,...,-1.0,15T,13T,5T,12T,9T,-1.0,-1.0,1,-1
9,-1,7T,-1,3,-1,5.0,-1,-1,10T,6,...,3.0,12.0,-1.0,-1.0,-1.0,3.0,-1.0,2.0,5.0,-1.0


In [32]:
top = top_20

rank_data_20 = pd.DataFrame()
    
for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a XGBoost Regressor model
    xgb_model = XGBRegressor(n_estimators=200,max_depth = 5, random_state=42, n_jobs=-1)
    xgb_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_xgb = xgb_model.predict(X_test)
    
    # Evaluate the model
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (XGBoost): {mse_xgb}')
    print(f'R-squared (XGBoost): {r2_xgb}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_xgb
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)
    
    if rank_data_20.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_20 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_20 = pd.concat([rank_data_20.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_20)

Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced', 'D-LEBRON_bball', 'DREB_nba', 'DRB', 'TRB', 'DEF WS_nba', 'DRPM_espn', 'RPM_espn', 'predator_defense_raptor', 'GS', 'mp_raptor']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 16.110317865729826
R-squared (XGBoost): -1.588330175861226


,Player_name,Rank,Pos,Points_won,Predicted_points
13740,Hakeem Olajuwon,-1,C,0.0,72.997696
13764,Gary Payton,2,PG,37.0,72.997696
13894,Dikembe Mutombo,1,C,39.0,72.997696
13836,Michael Jordan,4,SG,6.0,8.998991
13631,Derrick McKey,-1,SF,0.0,2.999332
13852,Patrick Ewing,-1,C,0.0,2.999332
13705,Shawn Kemp,-1,PF,0.0,2.999332
13783,Mario Elie,-1,SF,0.0,2.772643
13675,Nate McMillan,-1,SG,0.0,2.772643
13555,Latrell Sprewell,-1,SG,0.0,2.772643


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced', 'D-LEBRON_bball', 'DREB_nba', 'DRB', 'TRB', 'DEF WS_nba', 'DRPM_espn', 'RPM_espn', 'predator_defense_raptor', 'GS', 'mp_raptor']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 11.134329100698373
R-squared (XGBoost): 0.34448013917690323


,Player_name,Rank,Pos,Points_won,Predicted_points
13269,Alonzo Mourning,1,C,89.0,18.861637
13434,David Robinson,4,C,3.0,8.585159
13397,Scottie Pippen,7T,SF,1.0,6.850540
13182,Dikembe Mutombo,2,C,10.0,5.963746
13445,Kevin Garnett,7T,PF,1.0,5.956293
13039,Hakeem Olajuwon,7T,C,1.0,5.932784
13237,Eddie Jones,7T,SF,1.0,5.931642
13121,Bo Outlaw,-1,PF,0.0,5.922621
13488,Tim Duncan,5T,PF,2.0,5.863754
13406,Dennis Rodman,-1,PF,0.0,5.825609


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced', 'D-LEBRON_bball', 'DREB_nba', 'DRB', 'TRB', 'DEF WS_nba', 'DRPM_espn', 'RPM_espn', 'predator_defense_raptor', 'GS', 'mp_raptor']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 4.922307424927083
R-squared (XGBoost): 0.49233491687794784


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,265.615540
1184,Nikola Jokić,-1.0,C,0.0,140.335709
1007,Joel Embiid,-1,C,0.0,56.881294
1046,Sekou Doumbouya,-1.0,PF,0.0,36.994858
992,Paul George,-1.0,PF,0.0,36.287937
813,Ricky Rubio,-1.0,PG,0.0,34.715576
1026,Myles Turner,-1.0,C,0.0,34.005630
610,Michael Porter Jr.,-1.0,SF,0.0,31.546164
762,Jaylen Brown,-1.0,SF,0.0,29.885241
933,Al Horford,9,C,3.0,29.630798


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced', 'D-LEBRON_bball', 'DREB_nba', 'DRB', 'TRB', 'DEF WS_nba', 'DRPM_espn', 'RPM_espn', 'predator_defense_raptor', 'GS', 'mp_raptor']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 248.26480171954833
R-squared (XGBoost): 0.41655748634631695


,Player_name,Rank,Pos,Points_won,Predicted_points
323,Brook Lopez,2.0,C,309.0,104.702499
59,Jaren Jackson Jr.,1.0,C,391.0,70.510880
137,Evan Mobley,3.0,PF,101.0,27.977709
206,Nic Claxton,9T,C,7.0,10.537200
357,Nikola Jokić,-1.0,C,0.0,9.569198
497,Joel Embiid,9T,C,7.0,8.534583
408,Jarrett Allen,-1,C,0.0,7.057152
176,Giannis Antetokounmpo,6.0,PF,14.0,6.542648
524,Bam Adebayo,5.0,C,18.0,6.501785
158,Jrue Holiday,7T,PG,8.0,4.426101


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,-1,1,1,-1,3T,1,2,-1,4,-1,...,1.0,10T,3,-1.0,1.0,1.0,1.0,1.0,3.0,2.0
1,2,4,3T,-1,1,-1,1,-1,4,13T,...,7.0,13T,1,-1.0,15T,2.0,11,3,-1.0,1.0
2,1,7T,7,8T,-1,4,-1,-1,13T,3,...,5,15T,5.0,2.0,6,-1,3.0,9.0,-1,3.0
3,4,2,-1,-1,-1,13T,-1,4,-1,1,...,4.0,4,2,1,3,-1,-1.0,-1,-1.0,9T
4,-1,7T,2,-1,-1,13T,7,5,1,8,...,8T,2,-1.0,-1,-1,-1.0,-1,-1,-1.0,-1.0
5,-1,7T,8T,1,-1,3,6,9,7,4,...,13T,8,11T,-1,14.0,6T,2,5.0,-1.0,9T
6,-1,7T,5T,1,2,6,10T,18T,-1,5,...,-1.0,-1,4.0,-1,-1.0,5.0,-1.0,-1.0,-1.0,-1
7,-1,-1,3T,-1,-1,5.0,-1,1,6,-1.0,...,13T,3.0,-1.0,3,-1,3.0,10.0,-1,-1.0,6.0
8,-1,5T,8T,5T,-1,11,-1,8,10T,2,...,-1.0,1,-1,-1.0,15T,4,-1.0,-1,-1.0,5.0
9,-1,-1,8T,8T,-1,8,10T,-1,3,-1,...,8T,5.0,-1,-1.0,-1,-1.0,-1,6.0,9,7T


In [33]:
top = top_30

rank_data_30 = pd.DataFrame()
    
for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a XGBoost Regressor model
    xgb_model = XGBRegressor(n_estimators=200,max_depth = 5, random_state=42, n_jobs=-1)
    xgb_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_xgb = xgb_model.predict(X_test)
    
    # Evaluate the model
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (XGBoost): {mse_xgb}')
    print(f'R-squared (XGBoost): {r2_xgb}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_xgb
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_30.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_30 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_30 = pd.concat([rank_data_30.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_30)

Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced', 'D-LEBRON_bball', 'DREB_nba', 'DRB', 'TRB', 'DEF WS_nba', 'DRPM_espn', 'RPM_espn', 'predator_defense_raptor', 'GS', 'mp_raptor', 'W_nba', 'poss_raptor', 'predator_total_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP_100_poss', 'STL_nba', 'DRtg_100_poss', 'raptor_total_raptor', '%BLK_nba']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 16.11018643448534
R-squared (XGBoost): -1.5883090597380685


,Player_name,Rank,Pos,Points_won,Predicted_points
13740,Hakeem Olajuwon,-1,C,0.0,72.997467
13894,Dikembe Mutombo,1,C,39.0,72.997467
13764,Gary Payton,2,PG,37.0,72.997467
13836,Michael Jordan,4,SG,6.0,8.998990
13705,Shawn Kemp,-1,PF,0.0,2.999332
13631,Derrick McKey,-1,SF,0.0,2.999332
13852,Patrick Ewing,-1,C,0.0,2.999332
13555,Latrell Sprewell,-1,SG,0.0,2.772643
13675,Nate McMillan,-1,SG,0.0,2.772643
13783,Mario Elie,-1,SF,0.0,2.772643


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced', 'D-LEBRON_bball', 'DREB_nba', 'DRB', 'TRB', 'DEF WS_nba', 'DRPM_espn', 'RPM_espn', 'predator_defense_raptor', 'GS', 'mp_raptor', 'W_nba', 'poss_raptor', 'predator_total_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP_100_poss', 'STL_nba', 'DRtg_100_poss', 'raptor_total_raptor', '%BLK_nba']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 11.51506476234386
R-squared (XGBoost): 0.32206479778766905


,Player_name,Rank,Pos,Points_won,Predicted_points
13269,Alonzo Mourning,1,C,89.0,17.980135
13434,David Robinson,4,C,3.0,11.159957
13397,Scottie Pippen,7T,SF,1.0,7.491916
13039,Hakeem Olajuwon,7T,C,1.0,6.378341
13182,Dikembe Mutombo,2,C,10.0,6.351693
13445,Kevin Garnett,7T,PF,1.0,6.308943
13237,Eddie Jones,7T,SF,1.0,6.264395
13121,Bo Outlaw,-1,PF,0.0,6.192556
13488,Tim Duncan,5T,PF,2.0,6.073135
13406,Dennis Rodman,-1,PF,0.0,5.844840


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced', 'D-LEBRON_bball', 'DREB_nba', 'DRB', 'TRB', 'DEF WS_nba', 'DRPM_espn', 'RPM_espn', 'predator_defense_raptor', 'GS', 'mp_raptor', 'W_nba', 'poss_raptor', 'predator_total_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP_100_poss', 'STL_nba', 'DRtg_100_poss', 'raptor_total_raptor', '%BLK_nba']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 5.567388521164174
R-squared (XGBoost): 0.42580409706705014


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,140.284927
1184,Nikola Jokić,-1.0,C,0.0,99.213417
1007,Joel Embiid,-1,C,0.0,69.474197
1026,Myles Turner,-1.0,C,0.0,43.593758
992,Paul George,-1.0,PF,0.0,31.945063
762,Jaylen Brown,-1.0,SF,0.0,24.187418
1275,Stephen Curry,-1.0,PG,0.0,23.561291
813,Ricky Rubio,-1.0,PG,0.0,23.124643
1046,Sekou Doumbouya,-1.0,PF,0.0,17.395880
933,Al Horford,9,C,3.0,15.648423


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced', 'D-LEBRON_bball', 'DREB_nba', 'DRB', 'TRB', 'DEF WS_nba', 'DRPM_espn', 'RPM_espn', 'predator_defense_raptor', 'GS', 'mp_raptor', 'W_nba', 'poss_raptor', 'predator_total_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP_100_poss', 'STL_nba', 'DRtg_100_poss', 'raptor_total_raptor', '%BLK_nba']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 276.5637088729777
R-squared (XGBoost): 0.35005274862719227


,Player_name,Rank,Pos,Points_won,Predicted_points
323,Brook Lopez,2.0,C,309.0,81.279678
59,Jaren Jackson Jr.,1.0,C,391.0,66.713135
206,Nic Claxton,9T,C,7.0,11.584300
357,Nikola Jokić,-1.0,C,0.0,6.567778
208,Walker Kessler,-1.0,C,0.0,5.526576
408,Jarrett Allen,-1,C,0.0,4.140267
497,Joel Embiid,9T,C,7.0,4.078805
197,Kawhi Leonard,-1,SF,0.0,3.699897
524,Bam Adebayo,5.0,C,18.0,3.583007
75,Myles Turner,-1.0,C,0.0,3.200574


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,-1,1,1,-1,3T,-1,2,-1,1,-1,...,1.0,4,3,-1.0,1.0,1.0,1.0,1.0,3.0,2.0
1,1,4,7,3,1,4,1,4,4,3,...,7.0,6.0,1,-1.0,15T,-1,3.0,-1,-1.0,1.0
2,2,7T,5T,-1,-1,1,-1,-1,4,4,...,5,1,5.0,3,6,2.0,11,3,-1,9T
3,4,7T,8T,-1,2,3,-1,5,-1,13T,...,4.0,10T,2,-1,-1.0,-1,-1.0,-1,-1.0,-1.0
4,-1,2,3T,8T,-1,13T,7,9,13T,8,...,13T,13T,-1.0,-1,3,6T,-1.0,9.0,-1.0,-1.0
5,-1,7T,-1,-1,-1,13T,6,18T,7,1,...,-1.0,-1,-1.0,2.0,-1,-1.0,10.0,5.0,-1.0,-1
6,-1,7T,2,1,-1,6,10T,-1,6,16T,...,3.0,8,11T,1,14.0,-1.0,-1,-1,-1.0,9T
7,-1,-1,-1,1,-1,8,-1,8,-1,5,...,13T,15T,-1,-1,-1.0,5.0,-1.0,-1.0,-1.0,-1
8,-1,5T,3T,8T,-1,5.0,-1,6.0,8,7,...,2.0,3.0,4.0,5T,-1,3.0,2,-1.0,-1.0,5.0
9,-1,-1,8T,2,-1,2,10T,-1,-1,24T,...,-1.0,-1.0,13T,5T,8.0,-1.0,-1.0,-1.0,9,-1.0


In [34]:
top = top_40

rank_data_40 = pd.DataFrame()
    
for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a XGBoost Regressor model
    xgb_model = XGBRegressor(n_estimators=200,max_depth = 5, random_state=42, n_jobs=-1)
    xgb_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_xgb = xgb_model.predict(X_test)
    
    # Evaluate the model
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (XGBoost): {mse_xgb}')
    print(f'R-squared (XGBoost): {r2_xgb}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_xgb
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_40.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_40 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_40 = pd.concat([rank_data_40.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_40)

Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced', 'D-LEBRON_bball', 'DREB_nba', 'DRB', 'TRB', 'DEF WS_nba', 'DRPM_espn', 'RPM_espn', 'predator_defense_raptor', 'GS', 'mp_raptor', 'W_nba', 'poss_raptor', 'predator_total_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP_100_poss', 'STL_nba', 'DRtg_100_poss', 'raptor_total_raptor', '%BLK_nba', 'MIN_nba', 'MPG_espn', 'STL', 'MP', 'DBPM_advanced', 'BLK%_advanced', 'DREB%_nba', 'BLK_100_poss', 'DRB_100_poss', 'DRB%_advanced']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 16.11018643448534
R-squared (XGBoost): -1.5883090597380685


,Player_name,Rank,Pos,Points_won,Predicted_points
13740,Hakeem Olajuwon,-1,C,0.0,72.997467
13894,Dikembe Mutombo,1,C,39.0,72.997467
13764,Gary Payton,2,PG,37.0,72.997467
13836,Michael Jordan,4,SG,6.0,8.998990
13705,Shawn Kemp,-1,PF,0.0,2.999332
13631,Derrick McKey,-1,SF,0.0,2.999332
13852,Patrick Ewing,-1,C,0.0,2.999332
13555,Latrell Sprewell,-1,SG,0.0,2.772643
13675,Nate McMillan,-1,SG,0.0,2.772643
13783,Mario Elie,-1,SF,0.0,2.772643


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced', 'D-LEBRON_bball', 'DREB_nba', 'DRB', 'TRB', 'DEF WS_nba', 'DRPM_espn', 'RPM_espn', 'predator_defense_raptor', 'GS', 'mp_raptor', 'W_nba', 'poss_raptor', 'predator_total_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP_100_poss', 'STL_nba', 'DRtg_100_poss', 'raptor_total_raptor', '%BLK_nba', 'MIN_nba', 'MPG_espn', 'STL', 'MP', 'DBPM_advanced', 'BLK%_advanced', 'DREB%_nba', 'BLK_100_poss', 'DRB_100_poss', 'DRB%_advanced']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 11.621800623509591
R-squared (XGBoost): 0.3157808559153379


,Player_name,Rank,Pos,Points_won,Predicted_points
13269,Alonzo Mourning,1,C,89.0,16.754984
13434,David Robinson,4,C,3.0,10.531198
13397,Scottie Pippen,7T,SF,1.0,5.970061
13237,Eddie Jones,7T,SF,1.0,4.889778
13182,Dikembe Mutombo,2,C,10.0,4.844544
13406,Dennis Rodman,-1,PF,0.0,4.817644
13488,Tim Duncan,5T,PF,2.0,4.723114
13345,Kendall Gill,-1,SF,0.0,4.554414
13039,Hakeem Olajuwon,7T,C,1.0,4.484660
13445,Kevin Garnett,7T,PF,1.0,4.480813


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced', 'D-LEBRON_bball', 'DREB_nba', 'DRB', 'TRB', 'DEF WS_nba', 'DRPM_espn', 'RPM_espn', 'predator_defense_raptor', 'GS', 'mp_raptor', 'W_nba', 'poss_raptor', 'predator_total_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP_100_poss', 'STL_nba', 'DRtg_100_poss', 'raptor_total_raptor', '%BLK_nba', 'MIN_nba', 'MPG_espn', 'STL', 'MP', 'DBPM_advanced', 'BLK%_advanced', 'DREB%_nba', 'BLK_100_poss', 'DRB_100_poss', 'DRB%_advanced']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 5.758103186724329
R-squared (XGBoost): 0.4061346274085953


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,170.699799
1184,Nikola Jokić,-1.0,C,0.0,83.393532
1007,Joel Embiid,-1,C,0.0,68.781029
992,Paul George,-1.0,PF,0.0,44.796478
1026,Myles Turner,-1.0,C,0.0,39.186165
813,Ricky Rubio,-1.0,PG,0.0,38.412029
1275,Stephen Curry,-1.0,PG,0.0,25.548605
762,Jaylen Brown,-1.0,SF,0.0,25.072216
1046,Sekou Doumbouya,-1.0,PF,0.0,22.395330
1111,Giannis Antetokounmpo,6.0,PF,58.0,14.578673


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced', 'D-LEBRON_bball', 'DREB_nba', 'DRB', 'TRB', 'DEF WS_nba', 'DRPM_espn', 'RPM_espn', 'predator_defense_raptor', 'GS', 'mp_raptor', 'W_nba', 'poss_raptor', 'predator_total_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP_100_poss', 'STL_nba', 'DRtg_100_poss', 'raptor_total_raptor', '%BLK_nba', 'MIN_nba', 'MPG_espn', 'STL', 'MP', 'DBPM_advanced', 'BLK%_advanced', 'DREB%_nba', 'BLK_100_poss', 'DRB_100_poss', 'DRB%_advanced']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 269.7972038669691
R-squared (XGBoost): 0.36595458675330583


,Player_name,Rank,Pos,Points_won,Predicted_points
323,Brook Lopez,2.0,C,309.0,84.435150
59,Jaren Jackson Jr.,1.0,C,391.0,68.791618
206,Nic Claxton,9T,C,7.0,11.161429
137,Evan Mobley,3.0,PF,101.0,10.303954
141,Nikola Vučević,-1.0,C,0.0,8.471882
357,Nikola Jokić,-1.0,C,0.0,7.817036
403,Zion Williamson,-1.0,PF,0.0,6.563553
532,Darius Garland,-1.0,PG,0.0,4.406616
396,Rudy Gobert,-1.0,C,0.0,4.301207
524,Bam Adebayo,5.0,C,18.0,4.287775


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,-1,1,1,-1,3T,4,2,-1,1,-1,...,1.0,4,3,-1.0,1.0,-1,1.0,1.0,3.0,2.0
1,1,4,7,-1,1,-1,1,-1,4,3,...,7.0,-1,5.0,-1.0,15T,1.0,11,3,-1.0,1.0
2,2,7T,5T,-1,-1,13T,-1,4,4,13T,...,5,10T,1,3,6,2.0,3.0,-1,-1,9T
3,4,7T,3T,1,-1,1,-1,5,-1,4,...,4.0,6.0,2,-1,-1,-1,-1.0,9.0,-1.0,3.0
4,-1,2,-1,-1,2,3,7,9,13T,8,...,13T,8,11T,-1,3,-1.0,10.0,-1,-1.0,-1.0
5,-1,-1,8T,8T,-1,13T,6,18T,7,1,...,13T,13T,-1.0,1,-1.0,5.0,-1.0,5.0,-1.0,-1.0
6,-1,5T,3T,1,3T,6,-1,-1,6,5,...,2.0,15T,-1.0,-1,14.0,6T,8T,-1.0,-1.0,-1.0
7,-1,-1,-1,2,-1,8,10T,8,-1,24T,...,3.0,1,13T,2.0,-1,-1.0,-1,-1,-1.0,-1.0
8,-1,7T,8T,-1,-1,2,10T,6.0,8,16T,...,-1.0,-1.0,-1.0,5T,-1.0,3.0,-1,-1.0,-1.0,-1.0
9,-1,7T,-1,-1,-1,5.0,-1,-1,-1,20.0,...,-1.0,3.0,-1.0,5T,-1.0,-1.0,2,-1,6.0,5.0


In [35]:
top = top_50

rank_data_50 = pd.DataFrame()
    
for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a XGBoost Regressor model
    xgb_model = XGBRegressor(n_estimators=200,max_depth = 5, random_state=42, n_jobs=-1)
    xgb_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_xgb = xgb_model.predict(X_test)
    
    # Evaluate the model
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (XGBoost): {mse_xgb}')
    print(f'R-squared (XGBoost): {r2_xgb}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_xgb
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_50.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_50 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_50 = pd.concat([rank_data_50.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_50)

Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced', 'D-LEBRON_bball', 'DREB_nba', 'DRB', 'TRB', 'DEF WS_nba', 'DRPM_espn', 'RPM_espn', 'predator_defense_raptor', 'GS', 'mp_raptor', 'W_nba', 'poss_raptor', 'predator_total_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP_100_poss', 'STL_nba', 'DRtg_100_poss', 'raptor_total_raptor', '%BLK_nba', 'MIN_nba', 'MPG_espn', 'STL', 'MP', 'DBPM_advanced', 'BLK%_advanced', 'DREB%_nba', 'BLK_100_poss', 'DRB_100_poss', 'DRB%_advanced', '%DREB_nba', 'PER_advanced', 'OPP PTS OFF TOV_nba', 'OPP PTS PAINT_nba', 'OPP PTS 2ND CHANCE_nba', 'MOV_misc', 'TOV', 'SRS_misc', 'PW_misc', 'W_misc']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 16.068408890386667
R-squared (XGBoost): -1.581596958899016


,Player_name,Rank,Pos,Points_won,Predicted_points
13740,Hakeem Olajuwon,-1,C,0.0,72.997322
13764,Gary Payton,2,PG,37.0,72.997322
13894,Dikembe Mutombo,1,C,39.0,72.997322
13836,Michael Jordan,4,SG,6.0,7.867687
13705,Shawn Kemp,-1,PF,0.0,3.045614
13631,Derrick McKey,-1,SF,0.0,2.866752
13555,Latrell Sprewell,-1,SG,0.0,2.866752
13852,Patrick Ewing,-1,C,0.0,2.866752
13783,Mario Elie,-1,SF,0.0,2.866752
13889,Tim Duncan,5T,PF,4.0,2.131721


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced', 'D-LEBRON_bball', 'DREB_nba', 'DRB', 'TRB', 'DEF WS_nba', 'DRPM_espn', 'RPM_espn', 'predator_defense_raptor', 'GS', 'mp_raptor', 'W_nba', 'poss_raptor', 'predator_total_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP_100_poss', 'STL_nba', 'DRtg_100_poss', 'raptor_total_raptor', '%BLK_nba', 'MIN_nba', 'MPG_espn', 'STL', 'MP', 'DBPM_advanced', 'BLK%_advanced', 'DREB%_nba', 'BLK_100_poss', 'DRB_100_poss', 'DRB%_advanced', '%DREB_nba', 'PER_advanced', 'OPP PTS OFF TOV_nba', 'OPP PTS PAINT_nba', 'OPP PTS 2ND CHANCE_nba', 'MOV_misc', 'TOV', 'SRS_misc', 'PW_misc', 'W_misc']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGB

,Player_name,Rank,Pos,Points_won,Predicted_points
13269,Alonzo Mourning,1,C,89.0,16.718937
13434,David Robinson,4,C,3.0,10.627157
13397,Scottie Pippen,7T,SF,1.0,5.936703
13182,Dikembe Mutombo,2,C,10.0,4.878954
13488,Tim Duncan,5T,PF,2.0,4.807866
13406,Dennis Rodman,-1,PF,0.0,4.787127
13237,Eddie Jones,7T,SF,1.0,4.759343
13039,Hakeem Olajuwon,7T,C,1.0,4.449674
13445,Kevin Garnett,7T,PF,1.0,4.445818
13121,Bo Outlaw,-1,PF,0.0,4.437658


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced', 'D-LEBRON_bball', 'DREB_nba', 'DRB', 'TRB', 'DEF WS_nba', 'DRPM_espn', 'RPM_espn', 'predator_defense_raptor', 'GS', 'mp_raptor', 'W_nba', 'poss_raptor', 'predator_total_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP_100_poss', 'STL_nba', 'DRtg_100_poss', 'raptor_total_raptor', '%BLK_nba', 'MIN_nba', 'MPG_espn', 'STL', 'MP', 'DBPM_advanced', 'BLK%_advanced', 'DREB%_nba', 'BLK_100_poss', 'DRB_100_poss', 'DRB%_advanced', '%DREB_nba', 'PER_advanced', 'OPP PTS OFF TOV_nba', 'OPP PTS PAINT_nba', 'OPP PTS 2ND CHANCE_nba', 'MOV_misc', 'TOV', 'SRS_misc', 'PW_misc', 'W_misc']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGB

...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,173.699692
1184,Nikola Jokić,-1.0,C,0.0,83.481880
1007,Joel Embiid,-1,C,0.0,68.022224
992,Paul George,-1.0,PF,0.0,41.679920
1026,Myles Turner,-1.0,C,0.0,39.719063
813,Ricky Rubio,-1.0,PG,0.0,35.644157
762,Jaylen Brown,-1.0,SF,0.0,25.447668
1046,Sekou Doumbouya,-1.0,PF,0.0,23.701841
1275,Stephen Curry,-1.0,PG,0.0,23.404575
1111,Giannis Antetokounmpo,6.0,PF,58.0,13.776433


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'DWS_advanced', 'Dpoy_votes', 'BLK_nba', 'war_total_raptor', 'BLK', 'WINS_espn', 'war_reg_season_raptor', 'VORP_advanced', 'WS_advanced', 'D-LEBRON_bball', 'DREB_nba', 'DRB', 'TRB', 'DEF WS_nba', 'DRPM_espn', 'RPM_espn', 'predator_defense_raptor', 'GS', 'mp_raptor', 'W_nba', 'poss_raptor', 'predator_total_raptor', 'raptor_defense_raptor', 'MP_advanced', 'MP_100_poss', 'STL_nba', 'DRtg_100_poss', 'raptor_total_raptor', '%BLK_nba', 'MIN_nba', 'MPG_espn', 'STL', 'MP', 'DBPM_advanced', 'BLK%_advanced', 'DREB%_nba', 'BLK_100_poss', 'DRB_100_poss', 'DRB%_advanced', '%DREB_nba', 'PER_advanced', 'OPP PTS OFF TOV_nba', 'OPP PTS PAINT_nba', 'OPP PTS 2ND CHANCE_nba', 'MOV_misc', 'TOV', 'SRS_misc', 'PW_misc', 'W_misc']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGB

,Player_name,Rank,Pos,Points_won,Predicted_points
323,Brook Lopez,2.0,C,309.0,95.162628
59,Jaren Jackson Jr.,1.0,C,391.0,70.062477
206,Nic Claxton,9T,C,7.0,15.163095
208,Walker Kessler,-1.0,C,0.0,6.994411
357,Nikola Jokić,-1.0,C,0.0,6.308789
176,Giannis Antetokounmpo,6.0,PF,14.0,5.579443
158,Jrue Holiday,7T,PG,8.0,5.003392
396,Rudy Gobert,-1.0,C,0.0,3.868449
408,Jarrett Allen,-1,C,0.0,3.856900
524,Bam Adebayo,5.0,C,18.0,3.146033


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,-1,1,1,-1,3T,4,1,-1,1,-1,...,1.0,-1,3,-1.0,6,1.0,1.0,1.0,3.0,2.0
1,2,4,7,1,1,-1,2,1,4,3,...,7.0,6.0,2,-1.0,15T,2.0,3.0,-1,-1.0,1.0
2,1,7T,5T,8T,-1,1,-1,-1,4,24T,...,5,10T,5.0,-1,3,-1.0,11,3,-1,9T
3,4,2,8T,2,-1,13T,7,4,7,13T,...,4.0,8,11T,1,8.0,-1,10.0,9.0,-1.0,-1.0
4,-1,5T,-1,1,2,3,-1,5,-1,6,...,13T,1,4.0,3,1.0,-1,2,-1,-1.0,-1.0
5,-1,-1,3T,-1,-1,13T,6,9,13T,4,...,13T,4,1,-1,14.0,6T,-1.0,-1,-1.0,6.0
6,-1,7T,3T,-1,-1,6,-1,18T,6,1,...,2.0,13T,-1.0,-1,-1,-1.0,-1.0,5.0,-1.0,7T
7,-1,7T,8T,-1,3T,5.0,-1,8,-1,8,...,8T,15T,-1.0,5T,-1.0,3.0,-1.0,-1,-1.0,-1.0
8,-1,7T,-1,3,-1,8,10T,6.0,8,5,...,-1.0,2,-1.0,-1.0,-1.0,5.0,6T,11T,-1.0,-1
9,5T,-1,-1,-1,-1,-1,10T,11,5,16T,...,6,-1.0,-1.0,-1.0,4.0,6T,-1,-1.0,6.0,5.0


In [36]:
rank_data_10.to_csv('XGB+Per_Period_rank_data_10.csv', index=False)
rank_data_20.to_csv('XGB+Per_Period_rank_data_20.csv', index=False)
rank_data_30.to_csv('XGB+Per_Period_rank_data_30.csv', index=False)
rank_data_40.to_csv('XGB+Per_Period_rank_data_40.csv', index=False)
rank_data_50.to_csv('XGB+Per_Period_rank_data_50.csv', index=False)